In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
%load_ext autoreload
%autoreload 2

from os.path import join

import numpy as np

from pyspark.sql import SQLContext

In [3]:
path_root = "/home/tlin/notebooks/data"
path_pickle_train = join(
    path_root, "cache/two_domain/split_data/train")
path_pickle_test = join(
    path_root, "cache/two_domain/split_data/test")
path_pickle_baseline_sim = join(
    path_root, "cache/two_domain/item_based_sim/base_sim")
path_pickle_extended_sim = join(
    path_root, "cache/two_domain/extend_sim/extendsim")
path_pickle_private_mapped_sim = join(
    path_root, "cache/two_domain/private_mapping/privatemap")
path_pickle_nonprivate_mapped_sim = join(
    path_root, "cache/two_domain/private_mapping/nonprivatemap")
path_pickle_userbased_alterEgo = join(
    path_root, "cache/two_domain/cross_sim/user_based_alterEgo")
path_pickle_itembased_alterEgo = join(
    path_root, "cache/two_domain/cross_sim/item_based_alterEgo")
path_pickle_alterEgo_sim = join(
    path_root, "cache/two_domain/cross_sim/targetdomain_sim")

In [7]:
train_dataRDD = sc.pickleFile(path_pickle_train).cache()
test_dataRDD = sc.pickleFile(path_pickle_test).cache()


alterEgo_sim = sc.pickleFile(path_pickle_alterEgo_sim).cache()

In [6]:
item_based_alterEgo = sc.pickleFile(path_pickle_itembased_alterEgo).cache()
user_based_alterEgo = sc.pickleFile(path_pickle_userbased_alterEgo).cache()

user_based_dict_bd = sc.broadcast(user_based_alterEgo.collectAsMap())
item_based_dict_bd = sc.broadcast(item_based_alterEgo.collectAsMap())

In [22]:
    def find_neighbor(dataRDD):
        """For the rating pair of sim_pairs, we switch the position of id,
            and do groupBy operation to obtain each iid's neighbor.
        Args:
            dataRDD: in the form of ((iid1, iid2), [sim, RS])*
        """
        def key_on_first_item(iters):
            for id_pair, info_pair in iters:
                id1, id2 = id_pair
                yield id1, [(id2, info_pair)]
                yield id2, [(id1, info_pair)]
        return dataRDD.mapPartitions(
            key_on_first_item).reduceByKey(lambda a, b: a + b)
    
neighbor_RDD = find_neighbor(alterEgo_sim).cache()

In [24]:
t = neighbor_RDD.take(1)[0]
t

('A2895XG3G7IVFC',
 [('A3DBTL1NY5YLN9', [0.013845680213197273, 0.0059760334411662723]),
  ('A2CGL6SJXR0A2D', [0.024120907566221095, 0.010107926571313684]),
  ('A1M7459CVV6MFB', [0.020570379089063201, 0.0087272485797873604]),
  ('A3OXG8EMA8SSLK', [0.029090909090909087, 0.011928582484488425]),
  ('AQZH7YTWQPOBE', [0.005705820143257513, 0.0024906692667934108]),
  ('A1F4RWI7IJKILN', [0.40909090909090906, 0.0353335741066369]),
  ('A2KGB52KQ55W25', [0.029090909090909087, 0.011928582484488425]),
  ('ATSWMII881F71', [0.29607826273189608, 0.031791916646305207]),
  ('A2C0ZXU0H53OAL', [0.041674580885649862, 0.017088454473782974]),
  ('A3JTG7LGHYD59B', [0.32907259085720875, 0.033591473171000419]),
  ('A757BMZILP9EY', [0.029090909090909087, 0.011928582484488425]),
  ('AJ402UU5V0J8D', [0.40909090909090906, 0.0353335741066369]),
  ('AFSC64DATRQTJ', [0.04824181513244219, 0.019781316190926778]),
  ('A2895XG3G7IVFC', [0.029090909090909091, 0.011928582484488429]),
  ('A2895XG3G7IVFC', [0.0290909090909090

In [33]:
        def get_max_replacement_selection(replacement_selection_list):
            """Get max replacement_selection (local)."""
            return sorted(
                replacement_selection_list, key=lambda x: - abs(x[1]))[0][1]

ids, lines = t

lines = sorted(lines, key=lambda x: - abs(x[1][0]))
replacement_selection_list = [(line[0], line[1][1]) for line in lines]

max_replacement_selection = get_max_replacement_selection(replacement_selection_list)

In [34]:
lines

[('A1F4RWI7IJKILN', [0.40909090909090906, 0.0353335741066369]),
 ('AJ402UU5V0J8D', [0.40909090909090906, 0.0353335741066369]),
 ('A2KGB52KQ55W25', [0.40909090909090906, 0.0353335741066369]),
 ('ATE7C88K9DM0O', [0.40909090909090906, 0.0353335741066369]),
 ('A1YW95XJJLTEH7', [0.40909090909090906, 0.0353335741066369]),
 ('ATE3NHAN11ZGK', [0.40909090909090906, 0.0353335741066369]),
 ('ATERE75GD2JYB', [0.40909090909090906, 0.0353335741066369]),
 ('A1YPEA5L2J8DT5', [0.40909090909090906, 0.0353335741066369]),
 ('A25JJHKM3JXGT2', [0.40909090909090906, 0.0353335741066369]),
 ('A1NKCHDR98W5HL', [0.40909090909090906, 0.0353335741066369]),
 ('A174RK5T6T34M3', [0.40909090909090906, 0.0353335741066369]),
 ('A20R0PTFDPDNGT', [0.40909090909090906, 0.0353335741066369]),
 ('ADC98LDK2XLKX', [0.40909090909090906, 0.0353335741066369]),
 ('A3HMJPNFJJ94XG', [0.40909090909090906, 0.0353335741066369]),
 ('A3V3WG3JAOUK3L', [0.40909090909090906, 0.0353335741066369]),
 ('AIXY1GT9NHKBV', [0.40909090909090906, 0.03

In [61]:
        from math import log, e
    
        def get_k_sim(lists):
            """get k'th similarity, where k=mapping_range."""
            return lists[10 - 1][1][0] \
                if len(lists) >= 10 else lists[-1][1][0]
            
        def get_w(pairs, M_RS, k_sim):
            """calculate w.
                note that there exists a condition where len(pairs) < k.
            """
            v = len(pairs)
            return min(
                k_sim,
                (2 * 10 * M_RS / 0.1) * log(
                    10 * (
                        v - 10) / 0.1, e)) \
                if v > 10 else k_sim
            
        def split_to_sets(lines, k_sim, w):
            """split original list to two sets based on its similarity."""
            C1 = [line for line in lines if line[1][0] >= k_sim - w]
            C0 = [line for line in lines if line not in C1]
            return [C1, C0]
        
        def calculate_prob(lines, replacement_selection_list):
            """calculate probability (not normalized)."""
            probs = []
            for line in zip(lines, replacement_selection_list):
                tmp = 1.0 * np.exp(
                    0.1 * line[0][1][0] / (
                        2 * 10 * line[1][1]))
                probs.append((line[0][0], tmp))
            return probs
        
        def decide_num_selection(probs):
            """choose k=`mapping_range` non-duplicate item from pairs,
                and deal with the case that k < # of non-zero in pairs:
                    avoid error in `np.random.choice`
            Args:
                probs: normalized probabilities
            """
            nnz = np.count_nonzero(map(lambda prob: prob[1], probs))
            return 10 \
                if nnz >= 10 else nnz
            
        def normalize_prob(probs):
            """normalize probability."""
            sum_prob = sum([prob[1] for prob in probs])
            normalized_prob = [(prob[0], prob[1] / sum_prob) for prob in probs]
            return normalized_prob

        def weighted_pick(weights, n_picks):
            """weighted random selection
            returns:
                n_picks random indexes.
                    the chance to pick the index i
                    is give by the weight weights[i].
            """
            if type(weights) is not list:
                return 0
            t = np.cumsum(weights)
            s = np.sum(weights)
            st = list(set(np.searchsorted(t, np.random.rand(n_picks) * s)))
            left = n_picks - len(st)
            new = []
            if left != 0:
                new += weighted_pick(weights, left)
            return st + new
            
k_sim = get_k_sim(lines)
w = get_w(lines, max_replacement_selection, k_sim)
splites = split_to_sets(lines, k_sim, w)
modified_lines = [(pair[0], [max(pair[1][0], pair[1][0] - w), pair[1][1]]) for pair in pairs]

probs = calculate_prob(modified_lines, replacement_selection_list)

normalized_probs = normalize_prob(probs)

num_selection = decide_num_selection(normalized_probs)

index_choice = weighted_pick(
    list(map(lambda line: line[1], normalized_probs)), 2)

index_choice

[56, 139]

In [ ]:
    def prepare_private_selection(lines, max_replacement_selection):
        """prepare the procedure of mapping.
        Args:
            lines: in the form of (iid, [sim, sensitivity])*
            M_RS: maximal replacement selection value among all pairs.
        """
        def get_k_sim(lines):
            """get k'th similarity, where k=mapping_range."""
            return lines[self.mapping_range - 1][1][0] \
                if len(lines) >= self.mapping_range else lines[-1][1][0]

        def split_to_sets(lines, k_sim, w):
            """split original list to two sets based on its similarity."""
            C1 = [line for line in lines if line[1][0] >= k_sim - w]
            C0 = [line for line in lines if line not in C1]
            return [C1, C0]

        def get_w(lines, max_replacement_selection, k_sim):
            """calculate w.
                note that there exists a condition where len(lines) < k.
            """
            count = len(lines)
            return min(
                k_sim,
                (2 * self.mapping_range * max_replacement_selection / self.privacy_epsilon) * log(
                    self.mapping_range * (
                        count - self.mapping_range) / self.rpo, e)) \
                if count > self.mapping_range else k_sim

        k_sim = get_k_sim(lines)
        w = get_w(pairs, max_replacement_selection, k_sim)
        splits = split_to_sets(lines, k_sim, w)
        modified_lines = [(line[0], [max(line[1][0], line[1][0] - w), line[1][1]]) for line in lines]
        return splits, modified_lines